In [1]:
%load_ext autoreload
%autoreload 2

In [29]:
from absbox.local.generic import Generic

test01 = Generic(
    "Bond Step Up"
    ,{"collect":["2022-05-01","2022-06-01"] 
      ,"pay":["2022-06-15","2022-07-15"] # next distribution date
      ,"payFreq":["DayOfMonth",20]
      ,"poolFreq":"MonthEnd"
      ,"stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]
      ,'issuanceStat':{"IssuanceBalance":10000}
     }
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"stepUp":("ladder","2022-12-01",0.01,"MonthEnd")
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.00
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]}
    ,[["CollectedCash","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,"Amortizing"
    )

In [30]:
from absbox import API
localAPI = API("http://localhost:8081",lang='english',check=False)
#localAPI = API("https://absbox.org/api/dev",lang='english',check=False)

✅Connected, local lib:0.20.2, server:0.22.8

In [31]:
r = localAPI.run(test01
               ,poolAssump = ("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                       ,None
                                       ,None)
               ,runAssump = [("pricing"
                              ,{"date":"2022-07-20"
                                ,"curve":[["2021-01-01",0.025]
                                         ,["2024-08-01",0.025]]})]
               ,read=True)

In [32]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2022-07-15,801.11,5.75,198.89,0.07,204.64,"[<PayInt:A1>, <PayPrin:A1>]"
2022-07-20,801.11,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2022-08-20,689.46,5.52,111.65,0.07,117.17,"[<PayInt:A1>, <PayPrin:A1>]"
2022-09-20,576.48,4.09,112.98,0.07,117.07,"[<PayInt:A1>, <PayPrin:A1>]"
2022-10-20,462.82,3.31,113.66,0.07,116.97,"[<PayInt:A1>, <PayPrin:A1>]"
2022-11-20,348.69,2.75,114.13,0.07,116.88,"[<PayInt:A1>, <PayPrin:A1>]"
2022-12-20,234.09,2.18,114.60,0.08,116.78,"[<PayInt:A1>, <PayPrin:A1>]"
2023-01-20,119.12,1.71,114.97,0.09,116.68,"[<PayInt:A1>, <PayPrin:A1>]"
2023-02-20,3.51,0.97,115.61,0.10,116.58,"[<PayInt:A1>, <PayPrin:A1>]"


In [33]:
r['accounts']['acc01']

,balance,change,memo
date,,,
2022-06-01,234.64,234.64,<PoolInflow:CollectedCash>
2022-06-01,234.64,0.00,<PoolInflow:CollectedCash>
2022-07-15,204.64,-30.00,<SeqPayFee:trusteeFee>
2022-07-15,198.89,-5.75,<PayInt:A1>
2022-07-15,0.00,-198.89,<PayPrin:A1>
...,...,...,...
2023-09-20,115.93,0.00,<SeqPayFee:trusteeFee>
2023-09-20,115.93,0.00,<PayInt:A1>
2023-09-20,115.93,0.00,<PayPrin:A1>


In [12]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2022-07-15,801.11,5.75,198.89,0.07,204.64,"[<PayInt:A1>, <PayPrin:A1>]"
2022-07-20,801.11,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2022-08-20,689.46,5.52,111.65,0.07,117.17,"[<PayInt:A1>, <PayPrin:A1>]"
2022-09-20,576.48,4.09,112.98,0.07,117.07,"[<PayInt:A1>, <PayPrin:A1>]"
2022-10-20,462.82,3.31,113.66,0.07,116.97,"[<PayInt:A1>, <PayPrin:A1>]"
2022-11-20,348.69,2.75,114.13,0.07,116.88,"[<PayInt:A1>, <PayPrin:A1>]"
2022-12-20,234.09,2.18,114.60,0.08,116.78,"[<PayInt:A1>, <PayPrin:A1>]"
2023-01-20,119.00,1.59,115.09,0.08,116.68,"[<PayInt:A1>, <PayPrin:A1>]"
2023-02-20,3.22,0.80,115.78,0.08,116.58,"[<PayInt:A1>, <PayPrin:A1>]"


In [17]:
20 / 365 *0.08* 348.69

1.528504109589041

In [19]:
10 / 365 *0.08* 348.69

0.7642520547945205

In [20]:
1.528504109589041+0.7642520547945205

2.2927561643835617